In [1]:
import sqlite3
import json
from datetime import datetime

In [ ]:
timeframe = '2015-01'
sql_transaction = []
# {} will be replaced by what is given inside format
connection = sqlite3.connect('{}.db'.format(timeframe))
# Used to initialise SQL
c = connection.cursor()

def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, 
    comment TEXT, subreddit TEXT, unix INT, score INT)""")

def format_data(data):
    data = data.replace("\n", " newlinechar ").replace("\r", " newlinechar ").replace('"', "'")
    return data

def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        #print("find_parent", e)
        return false

def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True

def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        #print("find_parent", e)
        return false
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
            connection.commit()
            sql_transaction = []
  
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

    
# If executed directly
if __name__ == '__main__':
    create_table()
    #video 3
    row_counter = 0
    paired_rows = 0
    # Opening file, uses buffering since it is a big file
    with open("E:\ML\OnlineProjects\Deep learning\Chatbot\RC_{}\RC_{}".format(timeframe, timeframe), buffering=1000) as f:
        for row in f:
            row_counter += 1
            # Loads function convert from json to python format dumps does the opposite
            row = json.loads(row)
            parent_id = row['parent_id']
            comment_id = row['name']
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
            subreddit = row['subreddit']            
            parent_data = find_parent(parent_id)
            
            if score >= 2:
                if acceptable(body):
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                else:
                    if parent_data:
                        sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                        paired_rows += 1
                    else:
                        sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
            
            if row_counter % 100000 == 0:
                print("Total rows read: {}, Paired rows: {}, Time: {}".format(row_counter, paired_rows, str(datetime.now())))             

Total rows read: 100000, Paired rows: 282, Time: 2017-12-12 12:11:36.555133
Total rows read: 200000, Paired rows: 783, Time: 2017-12-12 12:32:39.978477
Total rows read: 300000, Paired rows: 1391, Time: 2017-12-12 12:51:39.378391
Total rows read: 400000, Paired rows: 2037, Time: 2017-12-12 13:10:36.434451
Total rows read: 500000, Paired rows: 2713, Time: 2017-12-12 13:29:45.190336
Total rows read: 600000, Paired rows: 3440, Time: 2017-12-12 13:47:46.678800
Total rows read: 700000, Paired rows: 4293, Time: 2017-12-12 14:09:57.260470
Total rows read: 800000, Paired rows: 5240, Time: 2017-12-12 14:31:31.929177
Total rows read: 900000, Paired rows: 6207, Time: 2017-12-12 14:53:11.239119
Total rows read: 1000000, Paired rows: 7066, Time: 2017-12-12 15:10:48.120874
Total rows read: 1100000, Paired rows: 7990, Time: 2017-12-12 15:28:16.494157
Total rows read: 1200000, Paired rows: 8823, Time: 2017-12-12 15:46:20.467791
Total rows read: 1300000, Paired rows: 9703, Time: 2017-12-12 16:05:39.5890

Total rows read: 10500000, Paired rows: 106552, Time: 2017-12-14 01:31:26.802305
Total rows read: 10600000, Paired rows: 107589, Time: 2017-12-14 01:52:50.833329
